In [ ]:
# Imports

import tensorflow as tf
from tensorflow import keras, GradientTape
from keras import layers, Sequential
from keras.datasets import mnist
from keras.layers import Dense
from tensorflow import cast, float32
import numpy as np
from tensorflow import losses


2026-01-29 02:51:10.519613: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/usr

: 

In [ ]:
# Imports

import tensorflow as tf
from tensorflow import keras, GradientTape
from keras import layers, Sequential
from keras.datasets import mnist
from keras.layers import Dense
from tensorflow import cast, float32
import numpy as np
from tensorflow import losses

class GA_Weights():
    def __init__(self, loss_fn, eval_model,
                 crossover_std = 0.9,
                 mutation_std = 0.2,
                 alpha=0.1, 
                 beta=0.05,
                 pop_size=10,
                 generations=5):
        self.loss_fn = loss_fn
        self.eval_model = eval_model
        self.alpha = alpha
        self.beta = beta
        self.crossover_std = crossover_std
        self.mutation_std = mutation_std
        self.pop_size = pop_size
        self.generations = generations

    def fitness_fn(self, individual, x, y):
        self.eval_model.set_weights(individual)
        y_pred = self.eval_model(x, training=False)
        error = self.loss_fn(y, y_pred).numpy()
        return error


    # Mutacion del indivudo seleccionado desde el crossover
    def mutate(self, individual):
        W, b = individual
        W = W + np.random.randn(*W.shape) * self.mutation_std
        b = b + np.random.randn(*b.shape) * self.mutation_std
        return [W, b]

    def crossover(self, parent_a, parent_b):
        Wa, ba = parent_a
        Wb, bb = parent_b
        W = 0.5 * Wa + 0.5 * Wb
        b = 0.5 * ba + 0.5 * bb
        return [W, b]

    # Seleccion de un solo individuo por torneo
    def tournament_selecction(self, population, fitness_scores, tournament_size = 3):
        idx=np.random.choice(len(population), tournament_size, replace=False)
        best = idx[np.argmin(fitness_scores[idx])]
        return population[best]

    def call_best_weights_GA(self, x, y):
        W, b = self.eval_model.get_weights()

        population = [
            [W + np.random.randn(*W.shape)*0.1,
            b + np.random.randn(*b.shape)*0.1]
           for _ in range(self.pop_size)]

        for _ in range(self.generations):
            fitness_scores = np.array([
                self.fitness_fn(individual, x, y) for individual in population 
            ])

            new_population = [population[np.argmin(fitness_scores)]]
            while len(new_population) < self.pop_size: 
                parent_a = self.tournament_selecction(population, fitness_scores)
                parent_b = self.tournament_selecction(population, fitness_scores)
                
                if np.random.rand() < self.crossover_std:
                    child = self.crossover(parent_a, parent_b)
                else:
                    child = parent_a

                child = self.mutate(child)
                new_population.append(child)
            
            population = new_population

            best = population[np.argmin([
                self.fitness_fn(ind, x, y) for ind in population
            ])]
        return best

class myModel():
    def __init__(self, feature_extractor, classifier, batch_size=32):
        super(myModel, self).__init__()
        self.feature_extractor = feature_extractor
        self.classifier = classifier
        self.batch_size = batch_size

        self.optimizer = keras.optimizers.Adam()
        self.loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        self.acc_metric = keras.metrics.SparseCategoricalAccuracy(name="accuracy")
        
        self.GA_Weights = GA_Weights(
            loss_fn=self.loss_fn,
            eval_model=self.classifier)

                
    def create_dataset(self, x, y):
        return tf.data.Dataset.from_tensor_slices((x, y)).batch(self.batch_size)

    def train_model(self, x, y, epochs):
        dataset = self.create_dataset(x, y)

        for epoch in range(epochs):
            print(f"\nEpoch {epoch+1}")
            for xd, yd in dataset:
                with GradientTape() as tape:
                    features = self.feature_extractor(xd, training=True)
                    y_pred = self.classifier(features, training=False)
                    loss = self.loss_fn(yd, y_pred)
                gradients = tape.gradient(
                    loss, 
                    self.feature_extractor.trainable_variables
                )
                self.optimizer.apply_gradients(
                    zip(gradients, self.feature_extractor.trainable_variables)
                )

            features = self.feature_extractor(x, training=False)             
            
            best_weights = self.GA_Weights.call_best_weights_GA(features.numpy(), y)
            self.classifier.set_weights(best_weights) 

            preds = self.classifier(features, training=False)
            self.acc_metric.update_state(y, preds)
            
            print({
                "loss": float(loss.numpy()),
                "accuracy": float(self.acc_metric.result().numpy())
            })
    
    def predict(self, x):
        dataset = tf.data.Dataset.from_tensor_slices(x).batch(self.batch_size)
        preds = []

        for xb in dataset:
            features = self.feature_extractor(xb, training=False)
            logits = self.classifier(features, training=False)
            preds.append(tf.argmax(logits, axis=1))

        return tf.concat(preds, axis=0)



(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

cnn = Sequential([
        layers.Input(shape = (28, 28, 1)),
        layers.Conv2D(32, (3,3), activation='relu', padding='same'),
        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.GlobalAveragePooling2D()
])

mlp = Dense(10, activation='softmax', name='classifier')

training = myModel(cnn, mlp)
training.train_model(x_train, y_train, epochs=2)



Epoch 1


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/nn.py:717: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2026-01-29 02:52:33.026938: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2026-01-29 02:52:33.318907: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 6021120000 exceeds 10% of free system memory.
2026-01-29 02:52:34.504127: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 6021120000 exceeds 10% of free system memory.
2026-01-29 02:52:36.659430: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 6021120000 exceeds 10% of free system memory.
2026-01-29 02:52:38.541866: W external/local_xla/xla/tsl/framework/cpu_alloc

: 

: 

In [11]:
pred = training.predict(x_test)

2026-01-29 02:25:52.879822: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 2007040000 exceeds 10% of free system memory.
2026-01-29 02:25:53.683003: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 2007040000 exceeds 10% of free system memory.


In [ ]:
def train_model(epochs, dataset, loss_fn, optimizer, feature_extractor, classifier):
    for epoch in range(epochs):
        with GradientTape() as tape:
            features = feature_extractor(x, training=True)
            preds = classifier(features, training=False)
            loss = loss_fn(y, preds)
        training_vars = feature_extractor.trainable_variables
        gradients = tape.gradient(loss, training_vars)
    optimizer.apply_gradients(zip(gradients, training_vars))
    w = classifier.get_weights()[0]
    print(w.shape)
    ga_update(features.numpy(), y.numpy(), classifier)
    print(f"Epoch{epoch}")